In [1]:
import pandas as pd
from label_data import Labeler

In [2]:
data = Labeler(
    "NAS100_M10_201707030100_202209292350.csv", "200_200_totals.csv"
).run()
useful_columns = [x for x in data.columns if x not in ["date", "spread"]]
data = data[useful_columns]
data

,open,high,low,close,sma_high,sma_low,operable
0,5636.1,5637.1,5635.7,5636.7,5620.451733,5615.619059,1
1,5636.9,5638.4,5636.5,5638.2,5620.385891,5615.570050,1
2,5638.1,5638.7,5637.2,5637.7,5620.322772,5615.524752,1
3,5637.4,5638.7,5637.4,5638.5,5620.264356,5615.470545,1
4,5638.7,5638.9,5637.5,5638.0,5620.205198,5615.411634,1
...,...,...,...,...,...,...,...
183845,11171.9,11181.2,11169.2,11177.1,11326.389604,11298.129950,1
183846,11176.4,11178.6,11172.5,11174.4,11326.166089,11297.913861,1
183847,11174.6,11176.6,11166.6,11167.3,11325.906188,11297.666584,1
183848,11167.3,11172.8,11166.3,11172.3,11325.635891,11297.403465,1


In [3]:
import numpy as np
import pandas as pd

In [4]:
CANDLES_HISTORY = 20
def get_dataset(train_data):
    x_train = []
    for i in range(CANDLES_HISTORY, len(train_data)):
        x_train.append(train_data[i-CANDLES_HISTORY:i, :-1])
    return np.array(x_train)

In [5]:
from sklearn.preprocessing import MinMaxScaler
from check_code import check_split

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.values)
X = get_dataset(scaled_data)
Y = np.array(scaled_data[CANDLES_HISTORY:len(scaled_data), -1])

assert check_split(data, scaler, CANDLES_HISTORY, X, Y)



In [6]:
import math, random
from check_code import check_train_data_shape, check_total

PERCENTAGE_DATA = 0
assert check_train_data_shape(X, Y)

training_data_len = math.ceil(len(Y) * .80)
x_train = X[:training_data_len]
y_train = Y[:training_data_len]

zipped = list(zip(x_train, y_train))
random.shuffle(zipped)
zipped_len_train = math.ceil(len(zipped) - len(zipped) * PERCENTAGE_DATA)
zipped = zipped[:zipped_len_train]
x_train, y_train = zip(*zipped)

x_train = np.array(x_train)
y_train = np.array(y_train)

assert check_train_data_shape(x_train, y_train)

divition = training_data_len  # + (len(Y[training_data_len:]) / 5)
shrink = math.ceil(divition)
x_test = X[shrink:-1]
y_test = Y[shrink:-1]

zipped = list(zip(x_test, y_test))
random.shuffle(zipped)
zipped_len_test = math.ceil(len(zipped) - len(zipped) * PERCENTAGE_DATA)
zipped = zipped[:zipped_len_test]
x_test, y_test = zip(*zipped)

x_test = np.array(x_test)
y_test = np.array(y_test)

assert check_train_data_shape(x_test, y_test)
assert check_total(y_test, Y, PERCENTAGE_DATA, training_data_len, y_train, shrink)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential()
model.add(LSTM(
    units=50,
    return_sequences=True,
    input_shape=(X.shape[1], X.shape[-1])
))
model.add(Dropout(0.2))
model.add(LSTM(units=15, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=15, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=15, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

: 

: 